# Assignments 4 
## Image Caption with RNN LSTM Attention


+ RNN
![rnn](./images/rnn.png)
    + Forward
        + 作业里面实现RNN前向传播的话，整体无难度，就跟着示意图里面的计算流程做就可以了
        + 当前时间步的隐状态$h_(t) = \sigma( W_(h) * h_(t-1) + W_(x) * x + b)$
    + Backward
        + 手动RNN反向传播，一开始不熟练的话就最好在纸上把计算图画出来，推导回去
        + 在实现的时候要注意变量之间的维度
    + Time step
        + 加上时间步之后也就注意一下输入数据不同维度对应的含义，计算时注意灵活变化（reshape permute...），问题也不太大
 
+ WordEmbedding
    + 词嵌入层的主要作用就是将一个输入系列在单个时间步上的词语转换为一个向量 [N, T] -> [N, T, D] 
    + 输入为所有单词在词表中的序号，将该序号采用长度为D的向量表示，作业中的简化方法是生成一个大小为[V, D]的嵌入层，V表示单词序号，D表示单词的向量表示
       
        
+ LSTM
    + 这个网络在本次作业实现基本也是没有太大的问题，跟着提示和课程讲义走就ok
![lstm](./images/lstm.png)
![lstm1](./images/lstm1.png)
    

+ LSTM&Attention
    + 加入注意力机制之后，主要就是注意矩阵乘法时的维度对应问题
    + 作用就是特征与当前时间步隐状态进行一个相似度计算，判断当前应该更注意哪个地方的特征
    + 在image caption任务里的意义就是不同的输出顺序对应着原图不同的特征

+ 任务1 - Image Caption 图像描述
    + 简单来说就是输入一张图片，网络给出该图片的文字描述
    + 训练步骤
        + CNN网络负责抽取图像的Feature，适当映射变换后作为RNN部分的初始隐状态输入
        + RNN网络部分负责学习对特征的描述能力
    + **需要注意的问题**
        + 训练时，由于知道该图片对应的描述语句，所以可以直接把所有的词语送入到RNN中
        + 测试时，由于不知道图片对应的句子，所以只有等网络产生当前时间步的输出词语之后，才能将该词语作为下一个时间步的输入，使网络继续进行，直到输出结束词
        + 实现细节上，RNN与LSTM还好，就是一步一步按照时间往前计算；而Attention就需要每次都计算相似度，得到所需数值

## Transformer
第二个作业主要就是根据 "Attention is all you need" 这篇文章中描述的Transformer模型的架构，基于PyTorch框架进行复现
+ 模型架构图
![Transformer](./images/transformer.png)
+ Input Embedding
    + 输入嵌入层主要作用就是将输入序列中的每个词语转换为一个向量表示，可以用nn.Embedding方法实现
    + 参数为序列长度和需要转换成的向量长度
+ Positional Encoding
    + 位置编码层，主要负责对输入的数据加入顺序信息，使得模型可以知道输入词语之间的先后顺序
    + 一般采用sinusoid方法，就是位置p上的词向量的第2i/2i+1个位置的编码分别为
        + $PE_(pos, 2i) = sin(\frac{pos}{1000^(\frac{2i}{d_model})})$
        + $PE_(pos, 2i+1) = cos(\frac{pos}{1000^(\frac{2i}{d_model})})$
+ Encoder
    + 编码器部分由若干个块组成，每个块的计算顺序相同
        + Mutil-Head Self Attention -> Residual Connect + LayerNorm -> (Dropout) -> FeedForward Network -> Residual Connect + LayerNorm -> (Dropout) -> Encoder Output
        + 多头自注意力机制主要是学习输入信息内部之间的表示关系，进行全局建模，使得每个位置的元素之间都可以产生联系，多头的目的是为了学习到更多的模式
        + 残差连接与ResNet中原理一样，为了使得反向传播中梯度流保持稳定
        + Layer Normalization - 层归一化，因为在CNN中BN针对的维度是特征通道，而在NLP中该通道对应的是序列长度/时间步，众所周知NLP中输入序列长度不稳定，在这个维度上求均值和方差不稳定，所以在Transformer中采用了层归一化，针对单个输入序列求均值和方差
        + 前馈神经网络主要用于学习更多非线性的关系
    + 编码器经过N个块循环计算之后，得到最后的编码器输出
+ Decoder
    + 解码器也是由若干个解码器模块组成
        + 相比较于Encoder，主要不同的地方为Decoder的第一个注意力模块为带掩码的多头自注意力机制(Masked Multi-Head Self-Attention)
        + 这里的掩码就是对输入的序列在时间维度上进行Mask，
        + 同时解码器模块中第二个注意力计算时，Encoder的输出作为Attention的Key和Query，第一个注意力层的输出作为Query，这里也是为了使得输出语言学习到输入语言的内部联系
    + 同样也是经过N个块循环计算，得到最后的解码器输出
    + 将解码器输出送入到一个线性层进行维度变换，然后Softmax计算概率最大的输出词语

+ Mask Multi-Head Self-Attention
个人的一些理解，假设key，query，value的大小都为(N, T, D)
    + 首先query与key做批次矩阵乘法，得到大小为(N, T, T)的结果，这里每一行可以视为当前时间上的词向量与其他时间上的此向量的相似度
    + Mask的大小为(N, T, T)，在形式上为一个下三角矩阵，针对每一行mask掉**当前时间步之后的权重**，即mask(time, time:-1) = True
    + 将上面的相似度矩阵中需要mask的地方重新赋值为一个很大的负数，这样在softmax计算出注意力权重时，被mask掉的部分的权重就是很小的值，这样使得模型不会关注到超过当前时间的数据
    + 将得到注意力权重与value进行矩阵乘法(感觉就是加权平均)，得到最终的注意力分数，(N, T, T) x (N, T, D) = (N, T, D)
        + 以某一个序列样本为例，权重矩阵中每一行代表了某一个序列样本中第t个词向量与序列中其他词向量的关系权重，再与原始词向量进行矩阵乘法 - (n, i, T) x (n, T, j) = (n, i, j)，就得到了第n个序列中第i个词中第j维的加权向量值